In [1]:
from urllib.parse import urlparse
from collections import Counter
from tqdm import tqdm 
import pandas as pd
import pymysql, tldextract, json, os, ast

## Reading fingerprinting

In [15]:
fingerprinting_file = '../processed_data/fingerprinting/all_fingerprinting.csv'

data = pd.read_csv(fingerprinting_file)

print(f'Loaded {len(data)} fingerprinting instances')

Loaded 88547 fingerprinting instances


In [16]:
data

etld                                        current_url  \
0        www.glamira.ae                            https://www.glamira.ae/   
1       www.chrono24.ae  https://www.chrono24.ae/cartier/santos-dumont-...   
2       www.chrono24.ae  https://www.chrono24.ae/cartier/santos-dumont-...   
3       www.chrono24.ae   https://www.chrono24.ae/cartier/tank--mod186.htm   
4       www.chrono24.ae   https://www.chrono24.ae/cartier/tank--mod186.htm   
...                 ...                                                ...   
88542   www.resnexus.us  https://www.resnexus.us/bed-and-breakfast-soft...   
88543   www.resnexus.us  https://www.resnexus.us/bed-and-breakfast-soft...   
88544   www.resnexus.us               https://www.resnexus.us/integrations   
88545   www.resnexus.us               https://www.resnexus.us/integrations   
88546  www.othership.us                   https://www.othership.us/account   

      location          category fingerprinting_type caller_id  \
0          UAE     country_coded        canvas_image    n20159   
1          UAE     country_coded               audio      n469   
2          UAE     country_coded        canvas_image      n469   
3          UAE     country_coded               audio      n480   
4          UAE     country_coded        canvas_image      n480   
...        ...               ...                 ...       ...   
88542      USA  country_specific               audio    n17381   
88543      USA  country_specific        canvas_image    n17381   
88544      USA  country_specific               audio    n24302   
88545      USA  country_specific        canvas_image    n24302   
88546      USA  country_specific        canvas_image    n19222   

                                              caller_url  \
0                https://www.glamira.ae/akam/13/4134d972   
1      https://challenges.cloudflare.com/cdn-cgi/chal...   
2      https://challenges.cloudflare.com/cdn-cgi/chal...   
3      https://challenges.cloudflare.com/cdn-cgi/chal...   
4      https://challenges.cloudflare.com/cdn-cgi/chal...   
...                                                  ...   
88542  https://assets.listenlayer.com/datalayer.min.j...   
88543  https://assets.listenlayer.com/datalayer.min.j...   
88544  https://assets.listenlayer.com/datalayer.min.j...   
88545  https://assets.listenlayer.com/datalayer.min.j...   
88546             https://m.stripe.network/out-4.5.45.js   

                                        caller_hash  \
0      z2QQLNF5VDJGBIzJyk8BGXblVCXID8MeNBahAGbcqvE=   
1      Ve4QJrGgmjH31xgEgI/oeMS2z2SpluppOOtYMXt6gxE=   
2      Ve4QJrGgmjH31xgEgI/oeMS2z2SpluppOOtYMXt6gxE=   
3      sToD8kL/09kj4/26sHb5kA891kvMAiGo7XsBYl1k4JE=   
4      sToD8kL/09kj4/26sHb5kA891kvMAiGo7XsBYl1k4JE=   
...                                             ...   
88542  xdMLQET1nInMhDAtLQAcHmNCMfImarR8LeGSYMJDuLc=   
88543  xdMLQET1nInMhDAtLQAcHmNCMfImarR8LeGSYMJDuLc=   
88544  xdMLQET1nInMhDAtLQAcHmNCMfImarR8LeGSYMJDuLc=   
88545  xdMLQET1nInMhDAtLQAcHmNCMfImarR8LeGSYMJDuLc=   
88546  8qijbHTFnPvnpz1EHP0YDKbmqZQqdO+bJAGR0cYFalk=   

                   caller_origin    caller_domain  \
0                 www.glamira.ae       glamira.ae   
1      challenges.cloudflare.com   cloudflare.com   
2      challenges.cloudflare.com   cloudflare.com   
3      challenges.cloudflare.com   cloudflare.com   
4      challenges.cloudflare.com   cloudflare.com   
...                          ...              ...   
88542     assets.listenlayer.com  listenlayer.com   
88543     assets.listenlayer.com  listenlayer.com   
88544     assets.listenlayer.com  listenlayer.com   
88545     assets.listenlayer.com  listenlayer.com   
88546           m.stripe.network   stripe.network   

                                          normalized_url  
0                                https://www.glamira.ae/  
1      https://www.chrono24.ae/cartier/santos-dumont-...  
2      https://www.chrono24.ae/cartier/santos-dumont-...  
3       https://www.chrono24.ae/ca

In [17]:
percentages = data['fingerprinting_type'].value_counts(normalize=True) * 100

# Optionally, round to 2 decimals
percentages = percentages.round(2)

print(percentages)

fingerprinting_type
matched_scripts    49.30
canvas_image       29.67
audio              15.75
webrtc              5.22
canvas_font         0.05
Name: proportion, dtype: float64


In [18]:
percentages = data[~(data['fingerprinting_type']=='matched_scripts')]['fingerprinting_type'].value_counts(normalize=True) * 100

percentages = percentages.round(2)

print(percentages)

fingerprinting_type
canvas_image    58.53
audio           31.06
webrtc          10.31
canvas_font      0.11
Name: proportion, dtype: float64


In [19]:
data.groupby('fingerprinting_type').count()

etld  current_url  location  category  caller_id  \
fingerprinting_type                                                      
audio                13944        13944     13944     13944      13944   
canvas_font             48           48        48        48         48   
canvas_image         26272        26272     26272     26272      26272   
matched_scripts      43657        43657     43657     43657      43657   
webrtc                4626         4626      4626      4626       4626   

                     caller_url  caller_hash  caller_origin  caller_domain  \
fingerprinting_type                                                          
audio                     12728        13944          12728          12728   
canvas_font                  30           48             30             30   
canvas_image              24011        26272          24011          24011   
matched_scripts           43657        43657          43657          43657   
webrtc                     4515         4626           4515           4515   

                     normalized_url  
fingerprinting_type                  
audio                         13944  
canvas_font                      48  
canvas_image                  26272  
matched_scripts               43657  
webrtc                         4626

### Reading sites tracking summary

In [20]:
sites_summary_file = '../processed_data/fingerprinting/sites_fingerprinting_summary.csv'

sites_summary_df = pd.read_csv(sites_summary_file)
sites_summary_df['distinct_fingerprinting_callers'] = sites_summary_df['distinct_fingerprinting_callers'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

sites_summary_df['distinct_third_party_fingerprinting_caller_domains'] = sites_summary_df['distinct_third_party_fingerprinting_caller_domains'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

sites_summary_df['distinct_third_party_fingerprinting_caller_origins'] = sites_summary_df['distinct_third_party_fingerprinting_caller_origins'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)



sites_summary_df

category location                          etld  \
0         country_coded      UAE                  www.tally.ae   
1         country_coded      UAE                     porter.ae   
2         country_coded      UAE                www.glamira.ae   
3         country_coded      UAE         www.hotelscombined.ae   
4         country_coded      UAE                  www.pearl.ae   
...                 ...      ...                           ...   
84230  country_specific      USA               www.resnexus.us   
84231  country_specific      USA            www.jgsc.k12.in.us   
84232  country_specific      USA       www.ci.harrisburg.or.us   
84233  country_specific      USA  www.paradiseindiancuisine.us   
84234  country_specific      USA              www.othership.us   

       is_fingerprinting  nb_fingerprinting_cases  \
0                  False                        0   
1                  False                        0   
2                   True                        1   
3                  False                        0   
4                  False                        0   
...                  ...                      ...   
84230               True                       10   
84231              False                        0   
84232              False                        0   
84233              False                        0   
84234               True                        1   

                         distinct_fingerprinting_callers  \
0                                                     []   
1                                                     []   
2              [https://www.glamira.ae/akam/13/4134d972]   
3                                                     []   
4                                                     []   
...                                                  ...   
84230  [https://assets.listenlayer.com/datalayer.min....   
84231                                                 []   
84232                                                 []   
84233                                                 []   
84234           [https://m.stripe.network/out-4.5.45.js]   

      distinct_third_party_fingerprinting_caller_domains  \
0                                                     []   
1                                                     []   
2                                                     []   
3                                                     []   
4                                                     []   
...                                                  ...   
84230                                  [listenlayer.com]   
84231                                                 []   
84232                                                 []   
84233                                                 []   
84234                                   [stripe.network]   

      distinct_third_party_fingerprinting_caller_origins  \
0                                                     []   
1                                                     []   
2                                                     []   
3                                                     []   
4                                                     []   
...                                                  ...   
84230                           [assets.listenlayer.com]   
84231                                                 []   
84232                                                 []   
84233                                                 []   
84234                                 [m.stripe.network]   

       is_first_party_fingerprinting  is_third_party_fingerprinting  \
0                              False                          False   
1                              False                          False   
2                               True                          False   
3                              False                          False   
4                              False                          False

## 1. Percentage of sites with fingerprinting

In [21]:
sites_summary_df['is_first_and_third_party_fingerprinting'] = (
    sites_summary_df['is_first_party_fingerprinting'] & sites_summary_df['is_third_party_fingerprinting']
)

# Group by category and location
summary = sites_summary_df.groupby(['category', 'location']).agg(
    s_sites=('is_fingerprinting', 'count'),  # total sites
    percent_fingerprinting=('is_fingerprinting', lambda x: (x.sum() / x.count() * 100).round(2)),
    percent_first_party_fingerprinting=('is_first_party_fingerprinting', lambda x: (x.sum() / x.count() * 100).round(2)),
    percent_third_party_fingerprinting=('is_third_party_fingerprinting', lambda x: (x.sum() / x.count() * 100).round(2)),
    percent_first_and_third_party_fingerprinting=('is_first_and_third_party_fingerprinting', lambda x: (x.sum() / x.count() * 100).round(2))
).reset_index()

# Optional: sort nicely
summary = summary.sort_values(['category', 'location']).reset_index(drop=True)

summary


category location  s_sites  percent_fingerprinting  \
0      country_coded  ALGERIA      206                    3.88   
1      country_coded  GERMANY     6181                   12.39   
2      country_coded    INDIA     5788                    8.57   
3      country_coded      UAE     1103                   20.31   
4      country_coded      USA      606                    5.94   
5   country_specific  ALGERIA     2434                    0.70   
6   country_specific  GERMANY     8460                    7.07   
7   country_specific    INDIA     8604                    5.04   
8   country_specific      UAE     8733                   10.90   
9   country_specific      USA     5308                    4.63   
10            global  ALGERIA     7461                   16.70   
11            global  GERMANY     7598                   23.41   
12            global    INDIA     7589                   15.81   
13            global      UAE     8354                   18.79   
14            global      USA     5810                   19.12   

    percent_first_party_fingerprinting  percent_third_party_fingerprinting  \
0                                 2.91                                0.97   
1                                 5.14                                8.14   
2                                 5.30                                4.22   
3                                11.79                                9.88   
4                                 1.82                                4.62   
5                                 0.25                                0.45   
6                                 2.85                                4.73   
7                                 3.95                                1.55   
8                                 5.84                                5.69   
9                                 1.73                                2.94   
10                                6.57                               11.38   
11                                7.84                               17.12   
12                                6.04                               10.61   
13                                6.70                               13.59   
14                                7.26                               12.81   

    percent_first_and_third_party_fingerprinting  
0                                           0.00  
1                                           0.89  
2                                           0.95  
3                                           1.36  
4                                           0.50  
5                                           0.00  
6                                           0.51  
7                                           0.45  
8                                           0.63  
9                                           0.04  
10                                          1.25  
11                                          1.55  
12                                          0.83  
13                                          1.50  
14                                          0.95

In [22]:
summary['percent_fingerprinting'] # IN THE PAPER

0      3.88
1     12.39
2      8.57
3     20.31
4      5.94
5      0.70
6      7.07
7      5.04
8     10.90
9      4.63
10    16.70
11    23.41
12    15.81
13    18.79
14    19.12
Name: percent_fingerprinting, dtype: float64

### Focus on global, with only etlds appearing on multiple locations only

In [ ]:
locations = sites_summary_df['location'].unique()
etlds_per_location = [set(sites_summary_df[sites_summary_df['location'] == loc]['etld']) for loc in locations]

common_etlds = set.intersection(*etlds_per_location)
print(f"Number of ETLDs common to all locations: {len(common_etlds)}")

# --- Step 2: Filter dataframe to only common ETLDs ---
df_common_etlds = sites_summary_df[sites_summary_df['etld'].isin(common_etlds)]

# --- Step 3: Compute tracking summary per location ---
summary_global = df_common_etlds.groupby(['category', 'location']).agg(
    s_sites=('is_fingerprinting', 'count'),
    percent_tracking=('is_fingerprinting', lambda x: round(x.sum() / x.count() * 100, 2)),
    percent_first_party_fingerprinting=('is_first_party_fingerprinting', lambda x: round(x.sum() / x.count() * 100, 2)),
    percent_third_party_fingerprinting=('is_third_party_fingerprinting', lambda x: round(x.sum() / x.count() * 100, 2)),
    percent_first_and_third_party_fingerprinting=('is_first_and_third_party_fingerprinting', lambda x: round(x.sum() / x.count() * 100, 2))
).reset_index()

# Optional: sort nicely
summary_global = summary_global.sort_values(['category', 'location']).reset_index(drop=True)

summary_global 

In [ ]:
summary_global['percent_tracking'] #IN THE PAPER

## 2. Number of different third-party tracking domains 

In [ ]:
def count_all_unique_domains(lists_of_domains):
    all_domains = set()
    for lst in lists_of_domains:
        if isinstance(lst, list):
            all_domains.update(lst)
        else:
            raise Exception('something wrong with the data')
    return len(all_domains)


    

In [ ]:
# Aggregate per category and location

third_party_stats_per_category_location = sites_summary_df.groupby(
    ['category', 'location']
)['distinct_third_party_fingerprinting_caller_origins'].agg(
    avg_third_party_domains_per_etld=lambda x: x.map(len).mean(),
    median_third_party_domains_per_etld=lambda x: x.map(len).median(),
    std_third_party_domains_per_etld=lambda x: x.map(len).std(),
    min_third_party_domains_per_etld=lambda x: x.map(len).min(),
    max_third_party_domains_per_etld=lambda x: x.map(len).max()
).reset_index()

# Compute total number of distinct third-party origins per category/location
number_distinct_origins = sites_summary_df.groupby(['category', 'location'])['distinct_third_party_fingerprinting_caller_origins'] \
    .apply(count_all_unique_domains) \
    .reset_index(name='number_distinct_third_party_fingerprinting_origins')

# Merge stats with total distinct counts
third_party_stats_per_category_location = third_party_stats_per_category_location.merge(
    number_distinct_origins,
    on=['category', 'location']
)

third_party_stats_per_category_location

In [ ]:
third_party_stats_per_category_location['number_distinct_third_party_fingerprinting_origins']

In [ ]:
# Aggregate per category and location

third_party_stats_per_category_location = sites_summary_df.groupby(
    ['category', 'location']
)['distinct_third_party_fingerprinting_caller_domains'].agg(
    avg_third_party_domains_per_etld=lambda x: x.map(len).mean(),
    median_third_party_domains_per_etld=lambda x: x.map(len).median(),
    std_third_party_domains_per_etld=lambda x: x.map(len).std(),
    min_third_party_domains_per_etld=lambda x: x.map(len).min(),
    max_third_party_domains_per_etld=lambda x: x.map(len).max()
).reset_index()

# Compute total number of distinct third-party origins per category/location
number_distinct_origins = sites_summary_df.groupby(['category', 'location'])['distinct_third_party_fingerprinting_caller_domains'] \
    .apply(count_all_unique_domains) \
    .reset_index(name='number_distinct_third_party_fingerprinting_domains')

# Merge stats with total distinct counts
third_party_stats_per_category_location = third_party_stats_per_category_location.merge(
    number_distinct_origins,
    on=['category', 'location']
)

third_party_stats_per_category_location

### Focus on global, with only etlds appearing on multiple locations only

In [ ]:
locations = sites_summary_df['location'].unique()
etlds_per_location = [set(sites_summary_df[sites_summary_df['location'] == loc]['etld']) for loc in locations]

common_etlds = set.intersection(*etlds_per_location)
print(f"Number of ETLDs common to all locations: {len(common_etlds)}")

# --- Step 2: Filter dataframe to only common ETLDs ---
df_common_etlds = sites_summary_df[sites_summary_df['etld'].isin(common_etlds)]

# --- Step 3: Aggregate third-party stats per category and location ---
third_party_stats_per_category_location = sites_summary_df.groupby(
    ['category', 'location']
)['distinct_third_party_fingerprinting_caller_domains'].agg(
    avg_third_party_domains_per_etld=lambda x: x.map(len).mean(),
    median_third_party_domains_per_etld=lambda x: x.map(len).median(),
    std_third_party_domains_per_etld=lambda x: x.map(len).std(),
    min_third_party_domains_per_etld=lambda x: x.map(len).min(),
    max_third_party_domains_per_etld=lambda x: x.map(len).max()
).reset_index()

number_distinct_domains = sites_summary_df.groupby(['category', 'location'])['distinct_third_party_fingerprinting_caller_domains'] \
    .apply(count_all_unique_domains) \
    .reset_index(name='number_distinct_third_party_fingerprinting_domains')

# Merge the results
third_party_stats_per_category_location = third_party_stats_per_category_location.merge(
    number_distinct_domains,
    on=['category', 'location']
)

# Optional: sort nicely
third_party_stats_per_category_location = third_party_stats_per_category_location.sort_values(
    ['category', 'location']
).reset_index(drop=True)

third_party_stats_per_category_location

### Priniting examples

In [ ]:
# --- Step 1: Focus on the "global" category ---
df_global = sites_summary_df[sites_summary_df['category'] == 'global'].copy()

# --- Step 2: Keep only eTLDs that exist in both UAE and INDIA ---
etlds_uae = set(df_global[df_global['location'] == 'UAE']['etld'])
etlds_india = set(df_global[df_global['location'] == 'INDIA']['etld'])
common_etlds_ui = etlds_uae.intersection(etlds_india)

df_common = df_global[df_global['etld'].isin(common_etlds_ui)]

# --- Step 3: Aggregate distinct third-party trackers per eTLD and location ---
def collect_unique_domains(series):
    """Collect all unique third-party domains for an ETLD and location."""
    all_domains = set()
    for lst in series.dropna():
        if isinstance(lst, list):
            all_domains.update(lst)
    return all_domains

df_trackers = (
    df_common.groupby(['location', 'etld'])['distinct_third_party_fingerprinting_caller_origins']
    .apply(collect_unique_domains)
    .reset_index(name='third_party_domains')
)

# --- Step 4: Compute counts ---
df_trackers['num_third_party_domains'] = df_trackers['third_party_domains'].apply(len)

# --- Step 5: Pivot so we can compare ---
pivot_counts = df_trackers.pivot(index='etld', columns='location', values='num_third_party_domains').fillna(0)
pivot_sets = df_trackers.pivot(index='etld', columns='location', values='third_party_domains').fillna({})

# --- Step 6: Filter on thresholds ---
mask = (pivot_counts['INDIA'] > 1) & (pivot_counts['UAE'] > 1)
selected_etlds = pivot_counts[mask].index

# --- Step 7: Build final comparison ---
results = []
for etld in selected_etlds:
    india_trackers = pivot_sets.loc[etld, 'INDIA']
    uae_trackers = pivot_sets.loc[etld, 'UAE']
    common_trackers = india_trackers & uae_trackers
    only_uae = uae_trackers - india_trackers
    only_india = india_trackers - uae_trackers

    results.append({
        'etld': etld,
        'num_india': len(india_trackers),
        'num_uae': len(uae_trackers),
        'num_common': len(common_trackers),
        'num_only_uae': len(only_uae),
        'num_only_india': len(only_india),
        'only_uae_trackers': sorted(list(only_uae)),
        'only_india_trackers': sorted(list(only_india)),
        'common_trackers': sorted(list(common_trackers))
    })

df_results = pd.DataFrame(results).sort_values(by='num_uae', ascending=False)

# --- Step 8: Display ---
print(f"Number of ETLDs where India > 5 and UAE > 11: {len(df_results)}\n")
pd.set_option('display.max_colwidth', None)
df_results.head(10)

In [ ]:
# Assume sites_summary_df has columns:
# 'category', 'location', 'distinct_third_party_tracking_domains'

# Filter by category (optional, e.g., 'global')
df_global = sites_summary_df[sites_summary_df['category'] == 'global']

# Collect sets of all third-party domains per country
domains_germany = set(domain for lst in df_global[df_global['location'] == 'GERMANY']['distinct_third_party_fingerprinting_caller_origins'] for domain in lst)
domains_india   = set(domain for lst in df_global[df_global['location'] == 'INDIA']['distinct_third_party_fingerprinting_caller_origins'] for domain in lst)
domains_uae     = set(domain for lst in df_global[df_global['location'] == 'UAE']['distinct_third_party_fingerprinting_caller_origins'] for domain in lst)

# Domains unique to Germany (not in India)
unique_to_germany = domains_germany - domains_india

# Domains unique to UAE (not in India)
unique_to_uae = domains_uae - domains_india

# Domains unique to India
unique_to_india = domains_india - (domains_germany | domains_uae)

# Domains shared between Germany and UAE but not India
shared_de_uae_not_india = (domains_germany & domains_uae) - domains_india

# Quick summary
print("Number of unique domains:")
print(f"Germany only (vs India): {len(unique_to_germany)}")
print(f"UAE only (vs India): {len(unique_to_uae)}")
print(f"India only: {len(unique_to_india)}")
print(f"Shared Germany+UAE, not India: {len(shared_de_uae_not_india)}")

# Optional: see the lists (sorted)
print("\nSample domains unique to Germany (5):", sorted(unique_to_germany)[:5])
print("Sample domains unique to UAE (5):", sorted(unique_to_uae)[:5])
print("Sample domains unique to India (5):", sorted(unique_to_india)[:5])
print("Sample domains shared Germany+UAE, not India (5):", sorted(shared_de_uae_not_india)[:5])


# 3. Most common third-party fingerprinters

In [ ]:
columns_data = {}

# Group by category and location
for (cat, loc), group in sites_summary_df.groupby(['category', 'location']):
    total_sites = group.shape[0]
    
    # Count domains
    domain_counts = Counter()
    for domains_list in group['distinct_third_party_fingerprinting_caller_origins']:
        for domain in set(domains_list):  # count each domain once per site
            domain_counts[domain] += 1
    
    # Compute percentage
    domain_percentages = {domain: count / total_sites * 100 for domain, count in domain_counts.items()}
    
    # Sort by percentage descending
    sorted_domains = sorted(domain_percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Format as "domain (xx%)"
    formatted_domains = [f"{domain} ({perc:.1f}%)" for domain, perc in sorted_domains]
    
    # Store in dict
    columns_data[f"{cat}-{loc}"] = formatted_domains

# Find max number of domains across all columns for consistent row length
max_len = max(len(domains) for domains in columns_data.values())

# Pad shorter columns with empty strings
for col, domains in columns_data.items():
    if len(domains) < max_len:
        domains.extend([''] * (max_len - len(domains)))

# Build DataFrame
df_top_domains = pd.DataFrame(columns_data)

df_top_domains.head(20)

In [ ]:
from collections import Counter
import pandas as pd

columns_data = {}

# Group only by location
for loc, group in sites_summary_df.groupby('location'):
    total_sites = group.shape[0]
    
    # Count distinct third-party origins per site
    domain_counts = Counter()
    for domains_list in group['distinct_third_party_fingerprinting_caller_origins']:
        for domain in set(domains_list):  # count each domain once per site
            domain_counts[domain] += 1
    
    # Compute percentage of sites per domain
    domain_percentages = {domain: count / total_sites * 100 for domain, count in domain_counts.items()}
    
    # Sort by prevalence (descending)
    sorted_domains = sorted(domain_percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Format as "domain (xx%)"
    formatted_domains = [f"{domain} ({perc:.1f}%)" for domain, perc in sorted_domains]
    
    # Store by location
    columns_data[loc] = formatted_domains

# Find max column length for consistent table
max_len = max(len(domains) for domains in columns_data.values())

# Pad shorter columns with empty strings
for loc, domains in columns_data.items():
    if len(domains) < max_len:
        domains.extend([''] * (max_len - len(domains)))

# Build final DataFrame
df_top_domains = pd.DataFrame(columns_data)

# Preview top 20
df_top_domains.head(20)


# 4. Most common fingerprinter organizations

In [ ]:
# Load the DuckDuckGo domain map
with open("utils/duckduckgo_domain_map.json") as f:
    radar_data = json.load(f)

def get_organization(domain):
    info = radar_data.get(domain) or {}
    return info.get('displayName', 'Unknown')


# Dictionary to hold top organizations per category-location
columns_data = {}

# Group by category and location
for (cat, loc), group in sites_summary_df.groupby(['category', 'location']):
    total_sites = group.shape[0]
    
    # Count organizations
    org_counts = Counter()
    for domains_list in group['distinct_third_party_fingerprinting_caller_domains']:
        # Get unique organizations per site
        organizations = {get_organization(domain) for domain in domains_list}
        for org in organizations:
            org_counts[org] += 1
    
    # Compute percentage
    org_percentages = {org: count / total_sites * 100 for org, count in org_counts.items()}
    
    # Sort by percentage descending
    sorted_orgs = sorted(org_percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Format as "organization (xx%)"
    formatted_orgs = [f"{org} ({perc:.1f}%)" for org, perc in sorted_orgs]
    
    # Store in dict
    columns_data[f"{cat}-{loc}"] = formatted_orgs

# Find max number of orgs across all columns for consistent row length
max_len = max(len(orgs) for orgs in columns_data.values())

# Pad shorter columns with empty strings
for col, orgs in columns_data.items(): 
    if len(orgs) < max_len:
        orgs.extend([''] * (max_len - len(orgs)))

# Build DataFrame
df_top_organizations = pd.DataFrame(columns_data)

df_top_organizations.head(20)


# 5. Country-specific fingerprinters

In [ ]:
# 5. Country-specific fingerprinters

import pandas as pd
from collections import Counter

# Assume sites_summary_df is already loaded
# Filter only rows with third-party tracking
df_tracking = sites_summary_df[sites_summary_df['is_third_party_fingerprinting']].copy()

# Get list of countries
countries = df_tracking['location'].unique()

# Prepare data
table_dict = {country: [] for country in countries}

for country in countries:
    df_country = df_tracking[df_tracking['location'] == country]
    df_other = df_tracking[df_tracking['location'] != country]
    
    # Collect all third-party origins in other countries
    origins_other = set()
    for lst in df_other['distinct_third_party_fingerprinting_caller_origins']:
        origins_other.update(lst)
    
    # Collect origins unique to this country
    origins_country_only = []
    for origins in df_country['distinct_third_party_fingerprinting_caller_origins']:
        for origin in origins:
            if origin not in origins_other:
                origins_country_only.append(origin)
    
    # Count frequency
    counter = Counter(origins_country_only)
    n_sites = len(df_country)
    
    # Compute percentage of sites
    table_dict[country] = [
        {"domain": origin, "pct_sites": count / n_sites * 100} 
        for origin, count in counter.most_common(20)
    ]

    print(country, len(counter))

# Convert to DataFrame safely (handles unequal lengths)
df_result = pd.concat(
    {country: pd.Series([f"{entry['domain']} ({entry['pct_sites']:.1f}%)" for entry in table_dict[country]]) 
     for country in countries},
    axis=1
)

# Display the resulting DataFrame
df_result


In [ ]:
import pandas as pd
from collections import Counter

# Assume df_summary is already loaded
# Filter only rows with third-party tracking
df_tracking = sites_summary_df[sites_summary_df['is_third_party_fingerprinting']].copy()

# Get list of countries
countries = df_tracking[df_tracking['category']=='global']['location'].unique()

# Prepare data
table_dict = {country: [] for country in countries}

for country in countries:
    df_country = df_tracking[df_tracking['location'] == country]
    df_other = df_tracking[df_tracking['location'] != country]
    
    # Collect all third-party origins in other countries
    origins_other = set()
    for lst in df_other['distinct_third_party_fingerprinting_caller_origins']:
        origins_other.update(lst)
    
    # Collect origins unique to this country
    origins_country_only = []
    for origins in df_country['distinct_third_party_fingerprinting_caller_origins']:
        for origin in origins:
            if origin not in origins_other:
                origins_country_only.append(origin)
    
    # Count frequency
    counter = Counter(origins_country_only)
    n_sites = len(df_country)
    
    # Compute percentage of sites
    table_dict[country] = [
        {"origin": origin, "pct_sites": count / n_sites * 100} 
        for origin, count in counter.most_common(10)
    ]

# Convert to DataFrame
df_result = pd.DataFrame({
    country: [f"{entry['origin']} ({entry['pct_sites']:.1f}%)" for entry in table_dict[country]] 
    for country in countries
})

# Display the resulting DataFrame
df_result


In [ ]:
import pandas as pd

# Assume 'sites_summary_df' contains your detailed third-party tracking info
# and has columns: 'location', 'etld', 'distinct_third_party_tracking_origins'

# Example: Find sites where "script.ioam.de" appears
target_origin = "www.google.ae"

# Filter rows where this origin is in the list of third-party origins
mask = sites_summary_df['distinct_third_party_tracking_origins'].apply(lambda lst: target_origin in lst)
df_origin_sites = sites_summary_df[mask]

# Select relevant columns
df_origin_sites_info = df_origin_sites[['location', 'etld']].drop_duplicates()

print(f"Websites (ETLDs) where {target_origin} appears:")
print(df_origin_sites_info)

# 6. Country specific organizations

In [ ]:
org_locations = {}

for (cat, loc), group in sites_summary_df.groupby(['category', 'location']):
    for domains_list in group['distinct_third_party_tracking_domains']:
        for domain in set(domains_list):
            org = get_organization(domain)
            if org not in org_locations:
                org_locations[org] = set()
            org_locations[org].add(loc)

# Step 2: Filter organizations that appear in only one location
country_specific_orgs = {org: list(locs)[0] for org, locs in org_locations.items() if len(locs) == 1}

# Step 3: Count percentage of sites per location
columns_data = {}
for loc in sites_summary_df['location'].unique():
    group = sites_summary_df[sites_summary_df['location'] == loc]
    total_sites = group.shape[0]
    
    org_counts = Counter()
    for domains_list in group['distinct_third_party_tracking_domains']:
        # Get unique organizations for this site
        organizations = {get_organization(domain) for domain in domains_list}
        for org in organizations:
            if org in country_specific_orgs and country_specific_orgs[org] == loc:
                org_counts[org] += 1
    
    # Compute percentages
    org_percentages = {org: count / total_sites * 100 for org, count in org_counts.items()}
    
    # Sort descending
    sorted_orgs = sorted(org_percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Format as "organization (xx%)"
    formatted_orgs = [f"{org} ({perc:.2f}%)" for org, perc in sorted_orgs]
    
    columns_data[loc] = formatted_orgs

# Step 4: Pad columns to equal length
max_len = max(len(orgs) for orgs in columns_data.values())
for col, orgs in columns_data.items():
    if len(orgs) < max_len:
        orgs.extend([''] * (max_len - len(orgs)))

# Step 5: Build DataFrame
df_country_specific_orgs = pd.DataFrame(columns_data)

df_country_specific_orgs.head(20)

# 7. Prevalence of the GTM in the first party context

In [ ]:
import re

def is_gtm(url):
    if pd.isna(url):
        return False
    url_lower = url.lower()
    return bool(re.search(r"gtm\.js|gtm\.start|googletagmanager", url_lower))


first_party_tracking_df['is_GTM'] = first_party_tracking_df['request_url'].apply(is_gtm)
first_party_tracking_df['is_GTM'].value_counts()


In [ ]:
# 1. Count total distinct eTLDs per category & location
total_sites = (
    first_party_tracking_df
    .groupby(['category', 'location'])['etld']
    .nunique()
    .reset_index(name='total_distinct_etlds')
)

# 2. Count distinct eTLDs with GTM (from previous step)
gtm_sites = (
    first_party_tracking_df[first_party_tracking_df['is_GTM']]
    .groupby(['category', 'location'])['etld']
    .nunique()
    .reset_index(name='distinct_etlds_with_GTM')
)

# 3. Merge totals with GTM counts
merged = pd.merge(gtm_sites, total_sites, on=['category', 'location'], how='right')

# 4. Fill NaN (if a group has zero GTM sites) with 0
merged['distinct_etlds_with_GTM'] = merged['distinct_etlds_with_GTM'].fillna(0)

# 5. Compute percentage
merged['percentage_GTM'] = (merged['distinct_etlds_with_GTM'] / merged['total_distinct_etlds']) * 100

# 6. Sort nicely
merged = merged.sort_values(['category', 'location']).reset_index(drop=True)

merged